In [2]:
import itertools

import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx

import meshplot as mp
import pyvista as pv
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from src import shapes

# Define The Figure and the Morse Function

In [3]:
def get_linear_morse(vector=None):
    if vector is None:
        vector = np.random.random(4)
    vector = np.array(vector)
    def f(points):
        return points @ vector
    return f

direction = np.random.random(3)
direction /= np.linalg.norm(direction)

In [4]:
f = lambda p: np.linalg.norm(p, axis=-1, ord=2)
f_linear = lambda p: p[:, 1]
#f = lambda p: (np.random.random(3)*p).sum(axis=-1)*(np.random.random(3)*p).sum(axis=-1)

In [5]:
def cylindrical_twist(vertices, k=1.0, mode="x", scale=1.0):
    """
    Nonlinear cylindrical twist diffeomorphism on R^3.

    vertices: (n,3) array
    mode:
      - "z": angle depends on z  (theta = k * tanh(z/scale))
      - "r": angle depends on radius r (theta = k * tanh(r/scale))
    k: twist strength (radians, roughly bounded by +/-k for tanh)
    scale: controls how quickly tanh saturates
    """
    if mode == 'x':
        v = vertices[:, [1, 2, 0]]
        v = cylindrical_twist(v, k=k, mode="z", scale=scale)
        v = v[:, [2, 0, 1]]
        return v
        
        
    v = vertices.copy()
    x, y, z = v[:, 0], v[:, 1], v[:, 2]

    r = np.sqrt(x*x + y*y)

    if mode == "z":
        theta = k * np.tanh(z / scale)
    elif mode == "r":
        theta = k * np.tanh(r / scale)
    else:
        raise ValueError('mode must be "z" or "r"')

    c, s = np.cos(theta), np.sin(theta)

    v[:, 0] = c * x - s * y
    v[:, 1] = s * x + c * y
    
    return v

In [60]:
n, m = 13, 12
vertices, faces = shapes.get_halftori_bouquet(leaves=2, n=n, m=m, l0=0.9, glue=False)

vertices = cylindrical_twist(vertices, k=-0.3, scale=1.5, mode='x')

vertices, faces = shapes.split_large_edges(vertices, faces, max_length=1.0)


print(f'faces.shape = {faces.shape}')

face_mean_values = f_linear(vertices[faces]).mean(axis=1)

p = mp.plot(vertices, faces, face_mean_values, shading={"wireframe": True})

faces.shape = (788, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [61]:
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3, dtype=faces.dtype), faces]).ravel()

mesh = pv.PolyData(vertices, faces_pv)
mesh.point_data["values"] = f_linear(vertices)  # per-vertex scalars

p = pv.Plotter(window_size=(600, 600))
p.add_mesh(
    mesh,
    scalars="values",
    cmap="viridis",
    smooth_shading=False,   # helps show linear interpolation nicely
    show_edges=True,      # set True if you want to see triangle edges
)
p.add_scalar_bar(title="values")
p.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772d681bd6a0_9&reconnect=auto" class="pyvi…

# Paths

In [62]:
from src.ms import MorseSmale

In [63]:
ms = MorseSmale(faces, f(vertices), vertices, forest_method='steepest')
#ms = MorseSmale(faces, f(vertices), vertices, forest_method='spaning')

paths = list(ms.iterate_paths())

In [64]:
from src import vis
from src.vis import plot_paths, plot_segmentation_forests, plot_ms_comparition


In [164]:
pl = plot_paths(ms)
pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772c601a2d20_32&reconnect=auto" class="pyv…

In [165]:
pl = plot_segmentation_forests(ms, plot_complex=True,  point_size=16)
pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772da08ff3b0_33&reconnect=auto" class="pyv…

In [65]:
ms0 =  MorseSmale(faces, f(vertices), vertices, forest_method='steepest')
ms1 =  MorseSmale(faces, f_linear(vertices), vertices, forest_method='steepest')

ms0.define_critical_points()
ms1.define_critical_points()

In [97]:
#ms =  MorseSmale(faces, f_linear(vertices), vertices, forest_method='steepest')
ms =  MorseSmale(faces, f(vertices), vertices, forest_method='steepest')

In [168]:
faces_components = ms.define_decomposition_by_paths()
pd.Series(faces_components).value_counts().sort_index()

0     98
1     98
2     98
3     98
4    100
5     98
6     98
7    100
Name: count, dtype: int64

In [169]:
new_mesh = vis.get_pv_mesh(ms.vertices, ms.faces)
new_mesh.cell_data['component'] = ms.define_decomposition_by_paths()


pl = pv.Plotter(window_size=(600, 600))
pl.add_mesh(new_mesh, scalars="component", cmap="rainbow", smooth_shading=False, show_edges=True, categories=True)
for path in ms.get_paths():
    pl.add_mesh(pv.lines_from_points(vertices[path]), color='white', line_width=4)

pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772bcd9bf8f0_36&reconnect=auto" class="pyv…

In [129]:
path_index = -2
path = ms.get_paths()[path_index]

pl = pv.Plotter(window_size=(600, 600))
pl.add_mesh(new_mesh, color="white", smooth_shading=False, show_edges=True, categories=True)
for chain in ms.get_paths():
    pl.add_mesh(pv.lines_from_points(vertices[chain]), color='midnightblue', line_width=6)


pl.add_mesh(pv.lines_from_points(vertices[path]), color='crimson', line_width=6)

pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772d87ac88f0_19&reconnect=auto" class="pyv…

In [130]:
def get_surrounding_faces(self: MorseSmale, chain, level=0):
    """
    """
    if level == 0:
        faces_vertex_permutations = self.faces[:, [list(perm) for perm in itertools.permutations(range(3), 2)]][..., None]
        chain_edges = np.array([chain[:-1], chain[1:]])
        surrounding_faces = np.argwhere((faces_vertex_permutations == chain_edges).all(axis=-2).any(axis=(-1, -2))).reshape(-1)
        return surrounding_faces
    surrounding_faces = get_surrounding_faces(self, chain, level=level-1)
    add_faces = np.unique([list(self.get_face_graph().neighbors(node)) for node in surrounding_faces])
    surrounding_faces = np.unique(np.concatenate([surrounding_faces, add_faces]))
    return add_faces

In [131]:
def get_surrounding_faces(self: MorseSmale, chain, level=0):
    """
    """
    faces_vertex_permutations = self.faces[:, [list(perm) for perm in itertools.permutations(range(3), 2)]][..., None]
    chain_edges = np.array([chain[:-1], chain[1:]])
    surrounding_faces0 = np.argwhere((faces_vertex_permutations == chain_edges).all(axis=-2).any(axis=(-1, -2))).reshape(-1)
    dist = nx.multi_source_dijkstra_path_length(self.get_face_graph(), sources=set(surrounding_faces0))
    surrounding_faces = np.array([key for key, value in dist.items() if value <= level])
    return surrounding_faces

In [132]:
def get_face_distances_from_chain(self: MorseSmale, chain):
    """
    """
    faces_vertex_permutations = self.faces[:, [list(perm) for perm in itertools.permutations(range(3), 2)]][..., None]
    chain_edges = np.array([chain[:-1], chain[1:]])
    surrounding_faces0 = np.argwhere((faces_vertex_permutations == chain_edges).all(axis=-2).any(axis=(-1, -2))).reshape(-1)
    dist = nx.multi_source_dijkstra_path_length(self.get_face_graph(), sources=set(surrounding_faces0))
    dist = np.array([dist[i] for i in range (self.n_faces)])

    return dist


In [133]:
def get_face_distances_from_chain(self: MorseSmale, chain):
    """
    """
    faces_vertex_permutations = self.faces[:, [list(perm) for perm in itertools.permutations(range(3), 2)]][..., None]
    chain_edges = np.array([chain[:-1], chain[1:]])
    surrounding_faces0 = np.argwhere((faces_vertex_permutations == chain_edges).all(axis=-2).any(axis=(-1, -2))).reshape(-1)
    dist = nx.multi_source_dijkstra_path_length(self.get_face_graph(), sources=set(surrounding_faces0))
    dist = np.array([dist[i] for i in range (self.n_faces)])
    return dist


In [134]:
def get_vertex_distances_from_chain(self: MorseSmale, chain):
    """
    """
    face_distances_from_chain = get_face_distances_from_chain(self, chain)
    dist = nx.multi_source_dijkstra_path_length(self.get_edge_graph(), sources=set(chain))
    dist = np.array([dist[i] for i in range (self.n_vertices)])
    return dist
    


In [135]:
def get_cuting_edges_surroounding_chain(self: MorseSmale, chain):
    """
    """
    graph01 = self.get_edge_graph()
    graph12 = self.get_face_graph()
    
    dists0 = get_vertex_distances_from_chain(self, chain)
    dists2 = get_face_distances_from_chain(self, chain)

    weights01 = {(e0, e1): dists0[[e0, e1]].max() for e0, e1 in graph01.edges()}
    weights12 = {(e0, e1): dists2[[e0, e1]].max() for e0, e1 in graph12.edges()}

    nx.set_edge_attributes(graph01, weights01, 'weight')
    nx.set_edge_attributes(graph12, weights12, 'weight')

    edges01 = np.array(list(nx.minimum_spanning_tree(graph01, weight='weight').edges))
    edges12 = np.array([data['intersection'] for e0, e1, data in nx.minimum_spanning_tree(graph12, weight='weight').edges(data=True)])
    
    edges = np.array(list(np.array(graph01.edges())))

    edges_to_cut = set(map(tuple, edges)) - set(map(tuple, edges01)) - set(map(tuple, edges12))
    edges_to_cut = np.array(list(edges_to_cut))
    return edges_to_cut

In [136]:
def get_cuting_edges_surroounding_chain(self: MorseSmale, chain):
    """
    """
    graph01 = self.get_edge_graph()
    graph12 = self.get_face_graph()
    
    dists0 = get_vertex_distances_from_chain(self, chain)
    dists2 = get_face_distances_from_chain(self, chain)

    weights01 = {(e0, e1): dists0[[e0, e1]].max() for e0, e1 in graph01.edges()}
    weights12 = {(e0, e1): dists2[[e0, e1]].min() for e0, e1 in graph12.edges()}

    nx.set_edge_attributes(graph01, weights01, 'weight')
    nx.set_edge_attributes(graph12, weights12, 'weight')

    tree01 = nx.minimum_spanning_tree(graph01, weight='weight')
    tree01_edges = set(map(tuple, tree01.edges()))

    graph12.remove_edges_from([(e0, e1) for e0, e1, data in graph12.edges(data=True) if tuple(data['intersection']) in tree01_edges])
    tree12 = nx.minimum_spanning_tree(graph12, weight='weight')
    tree12_edges = [data['intersection'] for e0, e1, data in nx.minimum_spanning_tree(tree12, weight='weight').edges(data=True)]
    tree12_edges = set(map(tuple, tree12_edges))


    edges = set(graph01.edges())
    edges_to_cut = set(map(tuple, edges)) - set(map(tuple, tree01_edges)) - set(map(tuple, tree12_edges))
    edges_to_cut = np.array(list(edges_to_cut))
    return edges_to_cut

In [137]:
def get_cuting_edges_surroounding_chain(self: MorseSmale, chain):
    """
    """
    graph0 = self.get_edge_graph()
    dists0 = get_vertex_distances_from_chain(self, chain)
    weights0 = {(e0, e1): dists0[[e0, e1]].max() for e0, e1 in graph0.edges()}
    nx.set_edge_attributes(graph0, weights0, 'weight')
    tree0 = nx.minimum_spanning_tree(graph0, weight='weight')
    tree0_edges = set(map(tuple, tree0.edges()))

    graph1 = self.get_face_graph()
    remove_cond = lambda e0, e1: ((e0, e1) in tree0_edges) or ((e1, e0) in tree0_edges)
    graph1.remove_edges_from([(e0, e1) for e0, e1, data in graph1.edges(data=True) if remove_cond(*data['intersection'])])
    dists1 = get_face_distances_from_chain(self, chain)
    weights1 = {(e0, e1): dists1[[e0, e1]].max() for e0, e1 in graph1.edges()}
    nx.set_edge_attributes(graph1, weights1, 'weight')
    tree1 = nx.minimum_spanning_tree(graph1, weight='weight')
    tree1_edges = set(map(tuple, [data['intersection'] for e0, e1, data in tree1.edges(data=True)]))

    edges = set(weights0.keys())

    return edges - tree1_edges - tree0_edges

In [138]:
def get_cuting_edges_surroounding_chain(self: MorseSmale, chain):
    """
    """
    def ordered_tuple(edge):
        a, b = edge
        return (a, b) if a <= b else (b, a)
    graph0 = self.get_edge_graph()
    dists0 = get_vertex_distances_from_chain(self, chain)
    weights0 = {(e0, e1): dists0[[e0, e1]].max() for e0, e1 in graph0.edges()}
    nx.set_edge_attributes(graph0, weights0, 'weight')
    tree0 = nx.minimum_spanning_tree(graph0, weight='weight')
    tree0_edges = set(map(ordered_tuple, tree0.edges()))

    #return tree0_edges

    graph1 = self.get_face_graph()
    graph1.remove_edges_from([(e0, e1) for e0, e1, data in graph1.edges(data=True) if ordered_tuple(data['intersection']) in tree0_edges])
    dists1 = get_face_distances_from_chain(self, chain)
    weights1 = {(e0, e1): dists1[[e0, e1]].max() for e0, e1 in graph1.edges()}
    nx.set_edge_attributes(graph1, weights1, 'weight')
    tree1 = nx.minimum_spanning_tree(graph1, weight='weight')

    tree1_edges = set(map(ordered_tuple, [data['intersection'] for e0, e1, data in tree1.edges(data=True)]))

    edges = set(map(ordered_tuple, weights0.keys()))

    initial_cut_edges = edges - tree1_edges - tree0_edges


    return initial_cut_edges

In [139]:
get_face_distances_from_chain(ms, path).shape

(788,)

In [140]:
get_vertex_distances_from_chain(ms, path).shape

(394,)

In [141]:
get_cuting_edges_surroounding_chain(ms, path)

{(63, np.int64(65)), (104, np.int64(383))}

In [142]:
cuting_graph = nx.Graph()
cuting_graph.add_edges_from(get_cuting_edges_surroounding_chain(ms, path))

In [143]:
nx.number_connected_components(cuting_graph)

2

In [144]:
len(nx.cycle_basis(cuting_graph))

0

In [145]:
{}

{}

In [146]:
new_mesh.point_data[f'Vertex distance to path {path_index}'] = get_vertex_distances_from_chain(ms, path)
new_mesh.cell_data[f'Face distance to path {path_index}'] = get_face_distances_from_chain(ms, path)

pl = pv.Plotter(shape=(1, 2), window_size=(1200, 600))

pl.subplot(0, 0)
pl.add_mesh(new_mesh, scalars=f'Vertex distance to path {path_index}', cmap="turbo", smooth_shading=False, show_edges=True, categories=False)
pl.add_mesh(pv.lines_from_points(vertices[path]), color='orangered', line_width=6)

pos = {node: val for node, val in enumerate(ms.vertices)}
vis.add_graph_to_plotter(pl, cuting_graph, pos, node_color='white', edge_color='white')

pl.subplot(0, 1)
pl.add_mesh(new_mesh, scalars=f'Face distance to path {path_index}', cmap="turbo", smooth_shading=False, show_edges=True, categories=False)
pl.add_mesh(pv.lines_from_points(vertices[path]), color='orangered', line_width=6)


pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772d87ac8620_20&reconnect=auto" class="pyv…

In [147]:
def get_surrounding_disk_cuts(self: MorseSmale, chain):
    """
    """
    pass

In [148]:
get_surrounding_disk_cuts(ms, path)

In [149]:
def compact_mesh(V, F):
    """
    Remove vertices not referenced by any face and reindex faces.
    Returns V2, F2, old2new, new2old.
    """
    V = np.asarray(V)
    F = np.asarray(F)

    used = np.zeros(len(V), dtype=bool)
    used[F.reshape(-1)] = True

    new2old = np.nonzero(used)[0]
    old2new = -np.ones(len(V), dtype=int)
    old2new[new2old] = np.arange(len(new2old))

    V2 = V[new2old]
    F2 = old2new[F]

    return V2, F2, old2new, new2old

In [150]:
from pygeodesic import geodesic

In [151]:
geo_paths = [ms.vertices[path]]
geo_distances = [np.linalg.norm(ms.vertices[path][1:] - ms.vertices[path][:-1], axis=1).sum()]

level = 1
while True:
    surrounding_faces = get_surrounding_faces(ms, path, level=level)

    V, F, old2new, new2old = compact_mesh(ms.vertices, ms.faces[surrounding_faces])
    source_vid = old2new[path[0]]
    target_vid = old2new[path[-1]]

    geo = geodesic.PyGeodesicAlgorithmExact(V, F)
    geo_distance, geopath = geo.geodesicDistance(source_vid, target_vid)

    if abs(geo_distances[-1] - geo_distance) <= 1e-6:
        break

    geo_paths.append(geopath)
    geo_distances.append(geo_distance)

    level += 1
    
geo_distances

[np.float64(3.0138125940118696), 3.011487258264056]

In [152]:
[geopath.shape for geopath in geo_paths]

[(7, 3), (18, 3)]

In [153]:
pl = pv.Plotter(window_size=(600, 600))
pl.add_mesh(new_mesh, color="white", smooth_shading=False, show_edges=True, categories=True)
for chain in ms.get_paths():
    pl.add_mesh(pv.lines_from_points(vertices[chain]), color='grey', line_width=6)

for i, geopath in enumerate(geo_paths):
    if geopath.size > 0:
        color = mcolors.to_hex(plt.get_cmap('inferno')(i/len(geo_paths)))
        pl.add_mesh(pv.lines_from_points(geopath), color=color, line_width=6)

pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772d87aca510_21&reconnect=auto" class="pyv…

In [158]:
ms0 =  MorseSmale(faces, f(vertices), vertices, forest_method='steepest')
ms1 =  MorseSmale(faces, f_linear(vertices), vertices, forest_method='steepest')

In [177]:
max_level = 6

geopath_sets0 = []
for path in ms0.get_paths():
    geo_paths = [ms0.vertices[path]]
    geo_distances = [np.linalg.norm(geo_paths[-1][1:] - geo_paths[-1][:-1], axis=1).sum()]
    level = 1
    while True:
        surrounding_faces = get_surrounding_faces(ms0, path, level=level)

        V, F, old2new, new2old = compact_mesh(ms0.vertices, ms0.faces[surrounding_faces])
        source_vid = old2new[path[0]]
        target_vid = old2new[path[-1]]

        geo = geodesic.PyGeodesicAlgorithmExact(V, F)
        geo_distance, geopath = geo.geodesicDistance(source_vid, target_vid)

        if abs(geo_distances[-1] - geo_distance) <= 1e-6:
            break

        geo_paths.append(geopath)
        geo_distances.append(geo_distance)

        level += 1
    geopath_sets0.append(geo_paths)



geopath_sets1 = []
for path in ms1.get_paths():
    geo_paths = [ms1.vertices[path]]
    geo_distances = [np.linalg.norm(geo_paths[-1][1:] - geo_paths[-1][:-1], axis=1).sum()]
    level = 1
    while True:
        surrounding_faces = get_surrounding_faces(ms1, path, level=level)

        V, F, old2new, new2old = compact_mesh(ms1.vertices, ms1.faces[surrounding_faces])
        source_vid = old2new[path[0]]
        target_vid = old2new[path[-1]]

        geo = geodesic.PyGeodesicAlgorithmExact(V, F)
        geo_distance, geopath = geo.geodesicDistance(source_vid, target_vid)

        if abs(geo_distances[-1] - geo_distance) <= 1e-6:
            break

        geo_paths.append(geopath)
        geo_distances.append(geo_distance)

        level += 1
    geopath_sets1.append(geo_paths)

In [209]:
pd.DataFrame({'Amount of geodesic variants for paths in case 0': pd.Series(map(len, geopath_sets0)).value_counts(), 
              'Amount of geodesic variants for paths in case 1': pd.Series(map(len, geopath_sets1)).value_counts()}).fillna(0).astype(int)

,Amount of geodesic variants for paths in case 0,Amount of geodesic variants for paths in case 1
2,12,4
4,0,4
7,4,0


In [210]:
pl = pv.Plotter(shape=(2, 2), window_size=(1200, 1000))

pl.subplot(0, 0)
vis.add_complex_to_plotter(pl, ms0, opacity=1.0, smooth_shading=False, show_edges=True,
                           with_values=True, data_title='values', color='white', value_cmap="viridis",
                           with_critical_points=True, min_color="lime", saddle_color="pink", max_color='orangered', point_size=12, 
                           with_paths=True, path_color='white', path_cmap=None, linewidth=4, eps=0.0)

pl.subplot(0, 1)
vis.add_complex_to_plotter(pl, ms1, opacity=1.0, smooth_shading=False, show_edges=True,
                           with_values=True, data_title='values', color='white', value_cmap="viridis",
                           with_critical_points=True, min_color="lime", saddle_color="pink", max_color='orangered', point_size=12, 
                           with_paths=True, path_color='white', path_cmap=None, linewidth=4, eps=0.0)


pl.subplot(1, 0)
pl.add_mesh(new_mesh, color="white", smooth_shading=False, show_edges=True, categories=True)
for geo_paths in geopath_sets0:
    for i, geopath in enumerate(geo_paths):
        if geopath.size > 0:
            color = mcolors.to_hex(plt.get_cmap('plasma')(i/(len(geo_paths) - 1)))
            pl.add_mesh(pv.lines_from_points(geopath), color=color, line_width=6)
vis.add_critical_points_to_plotter(pl, ms0)


pl.subplot(1, 1)
pl.add_mesh(new_mesh, color="white", smooth_shading=False, show_edges=True, categories=True)
for geo_paths in geopath_sets1:
    for i, geopath in enumerate(geo_paths):
        if geopath.size > 0:
            color = mcolors.to_hex(plt.get_cmap('plasma')(i/(len(geo_paths) - 1)))
            pl.add_mesh(pv.lines_from_points(geopath), color=color, line_width=6)
vis.add_critical_points_to_plotter(pl, ms1)

pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:33239/index.html?ui=P_0x772bad535610_66&reconnect=auto" class="pyv…

In [154]:
path

array([334, 327, 315, 291, 272, 245, 231])

In [155]:
ms.faces[(ms.faces[:, [list(perm) for perm in itertools.permutations(range(3), 2)]][..., None] == np.array([path[:-1], path[1:]])).all(axis=-2).any(axis=(-1, -2))]

array([[231, 245, 361],
       [231, 245, 362],
       [245, 272, 353],
       [245, 272, 354],
       [272, 291, 349],
       [272, 291, 350],
       [291, 315, 356],
       [291, 315, 359],
       [315, 327, 376],
       [315, 327, 379],
       [327, 334, 388],
       [327, 334, 391]])

In [156]:
def compact_mesh(V, F):
    """
    Remove vertices not referenced by any face and reindex faces.
    Returns V2, F2, old2new, new2old.
    """
    V = np.asarray(V)
    F = np.asarray(F)

    used = np.zeros(len(V), dtype=bool)
    used[F.reshape(-1)] = True

    new2old = np.nonzero(used)[0]
    old2new = -np.ones(len(V), dtype=int)
    old2new[new2old] = np.arange(len(new2old))

    V2 = V[new2old]
    F2 = old2new[F]

    return V2, F2, old2new, new2old

In [157]:
V2, F2, old2new, new2old = compact_mesh(vertices, surrounding)

start_idx = old2new[path[0]]
end_idx = old2new[path[-1]]

NameError: name 'surrounding' is not defined

In [ ]:
F2_pv = np.hstack([np.full((surrounding.shape[0], 1), 3, dtype=faces.dtype), F2]).ravel()

part_mesh = pv.PolyData(V2, F2_pv)

pl = pv.Plotter(window_size=(600, 600))
pl.add_mesh(part_mesh, color='white', smooth_shading=False, show_edges=True)
pl.show()

Widget(value='<iframe src="http://localhost:38733/index.html?ui=P_0x75cfb4a83bc0_4&reconnect=auto" class="pyvi…